# [Filtering VQE](https://arxiv.org/abs/2106.10055) - a Quantum Heuristic for solving VRP

***
### Authors: **Walid El Maouaki** and **Atharva Vidwans**
***

### **Theory**

The aim of this notebook is to implement the Filtering VQE algorithm for solving the VRP. We will not here explain all the algorithm, so just refer to [the original article](https://arxiv.org/abs/2106.10055) to understand it properly. Essentially, a regular VQE would use the expectation value of a hamiltonian $H$ on a quantum parametriced circuit in order to try and minimize that expectation value. What the Filtering VQE will do is use that parametriced circuit in order to approximate the action of some operator $F_\tau = f(H, \tau)$ that effectively projects a given wavefunction into lower energy eigenstates, thus, filtering the higher energy eigenstates.

Here, we will not cover how the VRP is formulated and solved in detail, since that is already exposed on [the Qiskit tutorial on the VRP](https://qiskit.org/documentation/tutorials/optimization/7_examples_vehicle_routing.html). However, we add an little summary of the used notation


---
Mathematically speaking, the vehicle routing problem (VRP) is a combinatorial problem, wherein the best routes from a depot to a number of clients and back to the depot are sought, given a number of available vehicles. There are a number of formulations possible, extending a number of formulations of the traveling salesman problem [Applegate et al, 2006]. Here, we present a formulation known as MTZ [Miller, Tucker, Zemlin, 1960]. 

Let $n$ be the number of clients (indexed as $1,\dots,n$), and $K$ be the number of available vehicles. Let $x_{ij} = \{0,1\}$ be the binary decision variable which, if it is $1$, activates the segment from node $i$ to node $j$. The node index runs from $0$ to $n$, where $0$ is (by convention) the depot. There are twice as many distinct decision variables as edges. For example, in a fully connected graph, there are $n(n+1)$ binary decision variables. 

If two nodes $i$ and $j$ have a link from $i$ to $j$, we write $i \sim j$. We also denote with $\delta(i)^+$ the set of nodes to which $i$ has a link, i.e., $j \in \delta(i)^+$ if and only if $i \sim j$. Similarly, we denote with 
$\delta(i)^-$ the set of nodes which are connected to $i$, in the sense that $j \in \delta(i)^-$ if and only if $j \sim i$. 
---



Thus, there will be a variable $x_{ij}$ that will tell if there is a vehicle going from node $i$ to node $j$. Our way to obtain a solution is to consider a vector ${\bf z}$ containing all that information

$$
{\bf z} = [x_{01},x_{02},\ldots,x_{10}, x_{12},\ldots,x_{n(n-1)}]^T,
$$

and, as $x_{ij}$ is a binary variable, that vector can be identified as a bitstring. This is why it makes sense to try and map the problem into a QUBO and then into an Ising Hamiltonian (it's diagonal); our solution can be depicted as one of the eigenstates that will, in fact, be one of the computational basis states.

In a very simple exercise, imagine that we are given the solution
$$
{\bf z} = [x_{01},x_{10}]^T = [1,1]^T,
$$
Thus, $x_{01}=1$ means that there is a vehicle going from the depot to node $1$ and $x_{10}=1$ means that there is a vehicle going from node $1$ to the depot. There are no more variables and that means that there is only $K=1$ vehicle visiting a single depot. Quite a trivial example.

#### **Importing standard Qiskit libraries and other necessary modules**

In [1]:
!pip install cplex 
# This are items I need to install and I had to make some changes to imports to get them to work.
!pip install qiskit 
!pip install qiskit.aqua.operators.expectations
!pip install qiskit_expectation
!pip install qiskit_optimization
!pip install .[dask]
!pip install dask_jobqueue

from time import process_time
import numpy as np

# Visualization tool
from qiskit.visualization import *
import matplotlib.pyplot as plt
import matplotlib.axes as axes

import itertools
import pandas as pd
import networkx as nx
import time
import math

from qiskit.algorithms.optimizers import ADAM, CG, GSLS, NELDER_MEAD, NFT, POWELL, SPSA, TNC, COBYLA, L_BFGS_B, SLSQP, AQGD, P_BFGS, GradientDescent
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, transpile, execute
from qiskit.utils  import QuantumInstance, algorithm_globals
from qiskit.circuit import ParameterVector
from qiskit.opflow import Z, I, X, Y, ListOp, PauliExpectation, CVaRExpectation, StateFn, CircuitSampler, CircuitStateFn, ListOp

# New Imports
from qiskit.providers.aer import AerSimulator
from concurrent.futures import ThreadPoolExecutor
from dask.distributed import LocalCluster, Client
from dask_jobqueue import LSFCluster
# End of new imports

from qiskit.algorithms import VQE, QAOA

import plotly.graph_objects as go

from qiskit import *
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
# from qiskit.aqua.operators.expectations import CVaRExpectation, PauliExpectation - Does not seem to work,  but has been replaced

import qiskit.tools.jupyter
# %qiskit_version_table

# Ignore Deprecation Warnings
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43.3 MB 17 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 7.0 MB/s 
     |████████████████████████████████| 18.0 MB 364 kB/s 
     |████████████████████████████████| 240 kB 52.3 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 112 kB 35.6 MB/s 
     |████████████████████████████████| 943 kB 45.5 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 58.7 MB/s 
     |████████████████████████████████| 49 kB 7.2 MB/s 
     |████████████████████████████████| 37.5 MB 1.3 MB/s 
     |████████████████████████████████| 4.1 MB 49.4 MB/s 
     |████████████████████████████████| 112 kB 27.1 MB/s 
  Created wheel for qiskit: filename=qiskit-0.37.1-py3-n

*** *italicized text*
### **Allocating Optimal Amount of Resources for Job - Currently Commented Out** ###
***

In [2]:
# Code for limiting requirements for this program based on device (not implemented yet)
# cluster = LSFCluster(cores=4, memory="4GB") # 4 CPU cores, 4GB of memory
# DASK_client = Client(cluster)
# backend = AerSimulator # Linking to the Aer Simulator
# backend.set_options(executor=my_dask_client, max_job_size=10) # Arbitrary number of jobs, but a large one just in case.

# This code is only for if one wants to implement their programs on their own devices, not on IBM's devices.
# To properly use this code, one has to remove the "backend =" code and replace it with just "backend"

In [3]:
# Code for dividing overall job into one circuit per job, to potentially lead to less errors and a faster runtime.
backend = AerSimulator()
# Setting an executor and a maximum job size (maximum number of circuits that can make up a job)
excute = ThreadPoolExecutor(max_workers=2)
backend.set_options(executor=excute)
backend.set_options(max_job_size=1) 
# Setting one job per circuit, as this allows each circuit to be run. This would theoretically allow for maximum accuracy as each circuit would have the minimum number of gates
# for the lowest possible error rate compared to running all circuits for one job (per layer), as in the current program.


## We have defined 3 classes:
**Class Initializer** 
- *GenerateInstance* : This method initializes the variables i.e. the number of nodes(including the depot)-n  and number of vehicles-K  

**Class ClassicalOptimizer**
- For a classical solution, we use IBM ILOG CPLEX. CPLEX is able to find the exact solution of this problem. We first define a ClassicalOptimizer class that encodes the problem in a way that CPLEX can solve, and then instantiate the class and solve it

**Class QuantumOptimizer**
- *binary_representation* : encodes the problem (M) into a QP terms (that’s basically linear algebra); 
- *construct_problem* : constructs a QUBO optimization problem as an instance of QuadraticProgram

In [4]:
# Classes for solving the VRP problem

class Initializer():

    def __init__(self, n, b):
        self.n = n
        self.b = b

    def generate_instance(self):

        n = self.n
        b = self.b
        
        # np.random.seed(33)
        np.random.seed(100*n + b)

        xc = (np.random.rand(n) - 0.5) * 50
        yc = (np.random.rand(n) - 0.5) * 50

        instance = np.zeros([n, n])
        for ii in range(0, n):
            for jj in range(ii + 1, n):
                instance[ii, jj] = (xc[ii] - xc[jj]) ** 2 + (yc[ii] - yc[jj]) ** 2
                instance[jj, ii] = instance[ii, jj]

        return xc, yc, instance
    
    
try:
    import cplex
    from cplex.exceptions import CplexError
except: 
    print("Warning: Cplex not found.")

class ClassicalOptimizer:

    def __init__(self, instance,n,K):

        self.instance = instance
        self.n = n  # number of nodes
        self.K = K  # number of vehicles


    def compute_allowed_combinations(self):
        f = math.factorial
        return f(self.n) / f(self.K) / f(self.n-self.K)


    def cplex_solution(self):

        # refactoring
        instance = self.instance
        n = self.n
        K = self.K

        my_obj = list(instance.reshape(1, n**2)[0])+[0. for x in range(0,n-1)]
        my_ub = [1 for x in range(0,n**2+n-1)]
        my_lb = [0 for x in range(0,n**2)] + [0.1 for x in range(0,n-1)]
        my_ctype = "".join(['I' for x in range(0,n**2)]) + "".join(['C' for x in range(0,n-1)])

        my_rhs = 2*([K] + [1 for x in range(0,n-1)]) + [1-0.1 for x in range(0,(n-1)**2-(n-1))] + [0 for x in range(0,n)]
        my_sense = "".join(['E' for x in range(0,2*n)]) + "".join(['L' for x in range(0,(n-1)**2-(n-1))])+"".join(['E' for x in range(0,n)])

        try:
            my_prob = cplex.Cplex()
            self.populatebyrow(my_prob,my_obj,my_ub,my_lb,my_ctype,my_sense,my_rhs)

            my_prob.solve()

        except CplexError as exc:
            print(exc)
            return

        x = my_prob.solution.get_values()
        x = np.array(x)
        cost = my_prob.solution.get_objective_value()

        return x,cost
    

    def populatebyrow(self,prob,my_obj,my_ub,my_lb,my_ctype,my_sense,my_rhs):

        n = self.n
    
        prob.objective.set_sense(prob.objective.sense.minimize)
        prob.variables.add(obj = my_obj, lb = my_lb, ub = my_ub, types = my_ctype)
    
        prob.set_log_stream(None)
        prob.set_error_stream(None)
        prob.set_warning_stream(None)
        prob.set_results_stream(None)

        rows = []
        for ii in range(0,n):
            col = [x for x in range(0+n*ii,n+n*ii)]
            coef = [1 for x in range(0,n)]
            rows.append([col, coef])

        for ii in range(0,n):
            col = [x for x in range(0+ii,n**2,n)]
            coef = [1 for x in range(0,n)]

            rows.append([col, coef])

        # Sub-tour elimination constraints:
        for ii in range(0, n):
            for jj in range(0,n):
                if (ii != jj)and(ii*jj>0):

                    col = [ii+(jj*n), n**2+ii-1, n**2+jj-1]
                    coef = [1, 1, -1]

                    rows.append([col, coef])

        for ii in range(0,n):
            col = [(ii)*(n+1)]
            coef = [1]
            rows.append([col, coef])

        prob.linear_constraints.add(lin_expr=rows, senses=my_sense, rhs=my_rhs)

        

class QuantumOptimizer:

    def __init__(self, instance, n, K):

        self.instance = instance
        self.n = n
        self.K = K

    def binary_representation(self,x_sol=0):

        instance = self.instance
        n = self.n
        K = self.K

        A = np.max(instance) * 100  # A parameter of cost function

        # Determine the weights w
        instance_vec = instance.reshape(n ** 2)
        w_list = [instance_vec[x] for x in range(n ** 2) if instance_vec[x] > 0]
        w = np.zeros(n * (n - 1))
        for ii in range(len(w_list)):
            w[ii] = w_list[ii]

        # Some variables I will use
        Id_n = np.eye(n)
        Im_n_1 = np.ones([n - 1, n - 1])
        Iv_n_1 = np.ones(n)
        Iv_n_1[0] = 0
        Iv_n = np.ones(n-1)
        neg_Iv_n_1 = np.ones(n) - Iv_n_1

        v = np.zeros([n, n*(n-1)])
        for ii in range(n):
            count = ii-1
            for jj in range(n*(n-1)):

                if jj//(n-1) == ii:
                    count = ii

                if jj//(n-1) != ii and jj%(n-1) == count:
                    v[ii][jj] = 1.

        vn = np.sum(v[1:], axis=0)

        # Q defines the interactions between variables
        Q = A*(np.kron(Id_n, Im_n_1) + np.dot(v.T, v))

        # g defines the contribution from the individual variables
        g = w - 2 * A * (np.kron(Iv_n_1,Iv_n) + vn.T) - \
                2 * A * K * (np.kron(neg_Iv_n_1, Iv_n) + v[0].T)

        # c is the constant offset
        c = 2 * A * (n-1) + 2 * A * (K ** 2)

        try:
            max(x_sol)
            # Evaluates the cost distance from a binary representation of a path
            fun = lambda x: np.dot(np.around(x), np.dot(Q, np.around(x))) + np.dot(g, np.around(x)) + c
            cost = fun(x_sol)
        except:
            cost = 0

        return Q, g, c, cost

    def construct_problem(self, Q, g, c) -> QuadraticProgram:
        qp = QuadraticProgram()
        for i in range(n * (n - 1)):
            qp.binary_var(str(i))
        qp.objective.quadratic = Q
        qp.objective.linear = g
        qp.objective.constant = c
        return qp
    


### Initialize the Classical Optimizer
- The below call initializes the classical optimizer to calcualte the Classical Cost and Classical solution. 
- Further, we also construct the Hamiltonian for given n and K values. 

In [5]:
### Problem definition
n= 2 # number of nodes + depot (n+1)
K= 1 # number of vehicles
b= 0

initializer = Initializer(n,b)
xc, yc, instance = initializer.generate_instance()

classical_optimizer = ClassicalOptimizer(instance,n,K)

x = None
z = None
try:
    x, classical_cost = classical_optimizer.cplex_solution()
    # Put the solution in the z variable
    z = [x[ii] for ii in range(n**2) if ii//n != ii%n]
    # Print the solution
except: 
    pass

algorithm_globals.massive=True
# Instantiate the quantum optimizer class with parameters: 
quantum_optimizer = QuantumOptimizer(instance, n, K)

try:
    if z is not None:
        Q, g, c, binary_cost = quantum_optimizer.binary_representation(x_sol = z)
    else:
        Q, g, c, binary_cost = quantum_optimizer.binary_representation()
except NameError as e:
    pass

qp = quantum_optimizer.construct_problem(Q, g, c)

quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'),
                                           seed_simulator=algorithm_globals.random_seed,
                                           seed_transpiler=algorithm_globals.random_seed)

vqe = VQE(quantum_instance=quantum_instance)
optimizer = MinimumEigenOptimizer(min_eigen_solver=vqe)
H, offset = optimizer._convert(qp, optimizer._converters).to_ising()
#print(H) ## Uncomment this line to view the Hamiltonian


In [6]:
def Ansatz(par, N, p=1):
    qc = QuantumCircuit(N)
    # Initial layer
    for i in range(N):
        qc.ry(par[i], i)
        
    for layer in range(p):
        par_counter = N+2*(N-1)*layer
    # Repeteable layer. To do the layer VQE thing, define this unit as a separate function
        for i in range(N//2):
            j = 2*i
            qc.cx(j, j+1)
            qc.ry(par[j+par_counter], j)
            qc.ry(par[j+1+par_counter], j+1)

        for i in range(N//2):
            j = 2*i
            if j+2<N: 
                qc.cx(j+1, j+2)
                qc.ry(par[N-N%2+j+1+par_counter], j+2)
                qc.ry(par[N-N%2+j+par_counter], j+1)
    return qc

We need to define some filtering operator. We will consider $F_\tau = \tau I - \mathcal{H}$ for a constant $\tau \ge E_\mathrm{max}$ since it's a very simple one that is indeed equivalent to a regular VQE. 

***

Next, we would have to define our objective function, which is

\begin{equation}
\begin{aligned}
\mathcal{C}_{t}(\boldsymbol{\theta}) &=\frac{1}{2} \||\psi(\boldsymbol{\theta})\rangle-\left|F_{t} \psi_{t-1}\right\rangle \|^{2} \\
&=1-\frac{\operatorname{Re}\left\langle\psi_{t-1}\left|F_{t}\right| \psi(\boldsymbol{\theta})\right\rangle}{\sqrt{\left\langle F_{t}^{2}\right\rangle_{\psi_{t-1}}}}
\end{aligned}
\end{equation}

However, given that the gradient descent update is written as

\begin{equation}
\boldsymbol{\theta}_{t}=\boldsymbol{\theta}_{t-1}-\left.\eta \sum_{j=1}^{m} \frac{\partial \mathcal{C}_{t}(\boldsymbol{\theta})}{\partial \theta_{j}}\right|_{\boldsymbol{\theta}_{t-1}} \boldsymbol{e}_{j},
\end{equation}

and we have an analytic expression for those derivatives

\begin{equation}
\left.\frac{\partial \mathcal{C}_{t}(\boldsymbol{\theta})}{\partial \theta_{j}}\right|_{\boldsymbol{\theta}_{t-1}}=-\frac{\left\langle F_{t}\right\rangle_{\psi_{t-1}^{j}}-\left\langle F_{t}\right\rangle_{\psi_{t-1}^{j-}}}{4 \sqrt{\left\langle F_{t}^{2}\right\rangle_{\psi_{t-1}}}},
\end{equation}

there is no need to evaluate that objective function $C_t(\theta)$.

***



In [7]:
def filtering_function6(E, c):
    return (c-E)

def bitstring_energy(H, bitstring):
    
    bitstring = bitstring[::-1]    
    #print(bitstring)
    spins = np.array([(-1)**(b == '1') for b in bitstring])
    #print(spins)
    value = 0
          
    for i, coef in enumerate(H.coeffs):
        #print("i is",i)
        weight = np.real(coef)
        indices = np.where(H.primitive.table.Z[i])
        #print("indices are:",indices[0])
        #print("------------")
        value += weight * np.prod(spins[indices])
    
    return value


def expectation(H,theta,n,l,shots,tau,filtering,power):
    
    circuit=Ansatz(theta,n,l)
    circuit.measure_all()
        
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()

    F_exp=0
    for bit,prob in counts.items():
        
        if filtering ==True:
            if power==True:
                F_exp+=(prob*filtering_function6(bitstring_energy(H, bit), tau)**2)
            else:
                F_exp+=(prob*filtering_function6(bitstring_energy(H, bit), tau))
        else:
            F_exp+=(prob*energies[i])

    return F_exp/shots

def gradient(op, n,l, shots,tau):
    def gradient_fn(params):
        grad_list=[]
        num_params = len(params)
        
        # Here, we apply the parameter shift-rule
        param_sets_to_eval = params + np.concatenate(
        (
            np.eye(num_params) * np.pi / 2,  # copy of the parameters with the positive shift
            -np.eye(num_params) * np.pi / 2,
        ),  # copy of the parameters with the negative shift
        axis=0,)
        
        denominator=4*np.sqrt(expectation(op,params,n,l,shots,tau,True,True))
        for i in range(len(params)):
            numerator=expectation(op,param_sets_to_eval[i],n,l,shots,tau,True,False)-expectation(op,param_sets_to_eval[i+len(params)],n,l,shots,tau,True,False)
            grad_list.append(numerator/denominator)
        return np.array(grad_list)
    return gradient_fn

def qiskit_expectation(op,n,l,shots):
    def q_exp(theta):
        psi = CircuitStateFn(Ansatz(theta,n,l))
        # define your backend or quantum instance (where you can add settings)
        backend = Aer.get_backend('qasm_simulator') 
        q_instance = QuantumInstance(backend, shots=shots)
        # define the state to sample
        measurable_expression = StateFn(op, is_measurement=True).compose(psi) 
        # convert to expectation value
        expectation = PauliExpectation().convert(measurable_expression)  
        # get state sampler (you can also pass the backend directly)
        sampler = CircuitSampler(q_instance).convert(expectation) 
        ##print('Math:', psi.adjoint().compose(op).compose(psi).eval().real)
        # evaluate
        return sampler.eval().real
    return q_exp

Callback function for cost values which later can be used for visualization purpose

In [8]:
F = c * (I^H.num_qubits) - H
fvqe_cost, fvqe_iter=[],[]
def callback(nfevs, x_next, cost, stepsize):
    fvqe_cost.append(cost)
    fvqe_iter.append(nfevs)

FVQE algorithm

In [9]:
shots=1024
num_qubits = H.num_qubits
num_para=2*(num_qubits-1) # The number of parameters in each Layer (except Layer 0).
c=9
layers=1

In [10]:
# Changes are made here in relation to errors that came up when implementing the original code
ss = process_time()
theta= np.asarray([*((np.pi/2)*np.ones(num_qubits)),*(np.zeros(2*(num_qubits-1)*layers))])
optimizer = GradientDescent(maxiter=80,learning_rate=1.0, callback=callback)

obj = qiskit_expectation(H,num_qubits,layers,shots) # <H>

grad_fun = gradient(F, num_qubits,layers, shots,c)

#result = optimizer.optimize(num_vars=num_qubits+2*(num_qubits-1)*layers, objective_function=obj,gradient_function=grad_fun, initial_point=theta)
result = optimizer.minimize(fun=obj, x0=theta,jac=grad_fun) # Revised version of the above lime with the same effect.
print(result.x) # Optimal parameters; creates a list of outputs
print(result.fun) # minimum parameter; result in a single output
print("DONE")
print("-----------------------")
print("\n*REPORT*\n")
qc = Ansatz(result.x, num_qubits, layers)
qc.measure_all()

# New code not related to the errors from the original code.
backend = AerSimulator()
# Setting an executor and a maximum job size (maximum number of circuits that can make up a job)
excute = ThreadPoolExecutor(max_workers=2)
backend.set_options(executor=excute)
backend.set_options(max_job_size=1) 
# result = backend.run(circ_list).result()
# End of new code

# backend = Aer.get_backend('qasm_simulator'); Removed because of the above groups of code.
job = execute(qc, backend, shots=shots)
res = job.result()
counts = res.get_counts()
revsol = counts.most_frequent()[::-1]
sol = np.array(list(revsol), dtype=int)
_,_,_,level = quantum_optimizer.binary_representation(x_sol=sol)
print("Quantum Cost is : ",level)
print("Classical Cost is : ",classical_cost)

[2.47761661 1.52794645 0.66754937 1.51923313]
-272409.287741662
DONE
-----------------------

*REPORT*

Quantum Cost is :  2737.781786348205
Classical Cost is :  2737.781786348361
